In [1]:
import os
import pandas as pd
from collections import OrderedDict
import numpy as np

`os.path.join(path, *paths)`<br>
Join one or more path components intelligently. The return value is the concatenation of path and any members of *paths with exactly one directory separator (os.sep) following each non-empty part except the last, meaning that the result will only end in a separator if the last part is empty. If a component is an absolute path, all previous components are thrown away and joining continues from the absolute path component.

### Country Code Concordances for Merging Records

In [2]:
codes = pd.read_csv('code-concordance.csv')

In [3]:
codes.head()

,WBcode,WBname,COWabb,COWcode,COWname,Income,MENA
0,AFG,Afghanistan,AFG,700.0,Afghanistan,Low,0
1,ALB,Albania,ALB,339.0,Albania,Upper-Middle,0
2,DZA,Algeria,ALG,615.0,Algeria,Upper-Middle,1
3,ADO,Andorra,AND,232.0,Andorra,High,0
4,AGO,Angola,ANG,540.0,Angola,Upper-Middle,0


### World Bank World Development Indicators (WDI)
https://data.worldbank.org/data-catalog/world-development-indicators

In [4]:
wdi = pd.read_csv(os.path.join('WDI', '7da42276-af1e-4ecf-ba32-04108473c2e7_Data.csv'))

In [5]:
wdi.head()

,Country Name,Country Code,Time,Time Code,"Net official flows from UN agencies, UNAIDS (current US$) [DT.NFL.UNAI.CD]",Net official aid received (current US$) [DT.ODA.OATL.CD],"Net bilateral aid flows from DAC donors, United States (current US$) [DC.DAC.USAL.CD]","Net bilateral aid flows from DAC donors, Total (current US$) [DC.DAC.TOTL.CD]",GDP (current US$) [NY.GDP.MKTP.CD],GDP growth (annual %) [NY.GDP.MKTP.KD.ZG],...,"Educational attainment, at least completed lower secondary, population 25+, male (%) (cumulative) [SE.SEC.CUAT.LO.MA.ZS]","Educational attainment, at least completed primary, population 25+ years, female (%) (cumulative) [SE.PRM.CUAT.FE.ZS]","Educational attainment, at least completed primary, population 25+ years, male (%) (cumulative) [SE.PRM.CUAT.MA.ZS]","Educational attainment, at least completed primary, population 25+ years, total (%) (cumulative) [SE.PRM.CUAT.ZS]",Incidence of HIV (% of uninfected population ages 15-49) [SH.HIV.INCD.ZS],Individuals using the Internet (% of population) [IT.NET.USER.ZS],"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate) [SL.TLF.CACT.FE.ZS]","Labor force participation rate, male (% of male population ages 15+) (modeled ILO estimate) [SL.TLF.CACT.MA.ZS]","Population, total [SP.POP.TOTL]","Unemployment, total (% of total labor force) (modeled ILO estimate) [SL.UEM.TOTL.ZS]"
0,Afghanistan,AFG,1960.0,YR1960,..,..,15000000,15670000,537777811.1,..,...,..,..,..,..,..,..,..,..,8996351,..
1,Afghanistan,AFG,1961.0,YR1961,..,..,30000000,33550000,548888895.6,..,...,..,..,..,..,..,..,..,..,9166764,..
2,Afghanistan,AFG,1962.0,YR1962,..,..,13000000,15260000,546666677.8,..,...,..,..,..,..,..,..,..,..,9345868,..
3,Afghanistan,AFG,1963.0,YR1963,..,..,32000000,34880000,751111191.1,..,...,..,..,..,..,..,..,..,..,9533954,..
4,Afghanistan,AFG,1964.0,YR1964,..,..,37000000,44220000,800000044.4,..,...,..,..,..,..,..,..,..,..,9731361,..


In [6]:
wdi_mortality = pd.read_csv(os.path.join('WDI', '14f3b5cc-5125-4012-9b9c-517473664bf8_Data.csv'))

In [7]:
wdi_mortality.columns

Index(['Country Name', 'Country Code', 'Time', 'Time Code',
       'Maternal mortality ratio (modeled estimate, per 100,000 live births) [SH.STA.MMRT]',
       'Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]',
       'Life expectancy at birth, female (years) [SP.DYN.LE00.FE.IN]',
       'Life expectancy at birth, male (years) [SP.DYN.LE00.MA.IN]',
       'Life expectancy at birth, total (years) [SP.DYN.LE00.IN]'],
      dtype='object')

In [8]:
wdi_mortality.head()

,Country Name,Country Code,Time,Time Code,"Maternal mortality ratio (modeled estimate, per 100,000 live births) [SH.STA.MMRT]","Mortality rate, infant (per 1,000 live births) [SP.DYN.IMRT.IN]","Life expectancy at birth, female (years) [SP.DYN.LE00.FE.IN]","Life expectancy at birth, male (years) [SP.DYN.LE00.MA.IN]","Life expectancy at birth, total (years) [SP.DYN.LE00.IN]"
0,Afghanistan,AFG,1960.0,YR1960,..,245.7,33.133,31.58,32.33756098
1,Afghanistan,AFG,1961.0,YR1961,..,241.2,33.586,32.026,32.78697561
2,Afghanistan,AFG,1962.0,YR1962,..,236.9,34.03,32.466,33.22892683
3,Afghanistan,AFG,1963.0,YR1963,..,232.7,34.469,32.903,33.66690244
4,Afghanistan,AFG,1964.0,YR1964,..,228.5,34.904,33.338,34.10190244


In [9]:
del wdi['Country Name']
del wdi['Time Code']
wdi = wdi.merge(wdi_mortality, on=['Country Code', 'Time'], how='left')

In [10]:
#map abbreviated col names to long col descripts
wdi_col_map = OrderedDict()
for col in wdi.columns:
    if '[' in col:
        col_parts = col.split('[')
        wdi_col_map[col_parts[1].strip(']')] = col_parts[0].strip()
    else:
        wdi_col_map['.'.join(col.split())] = col

In [11]:
wdi.columns = wdi_col_map.keys()
wdi.replace('..', np.nan, inplace=True)

In [12]:
wdi.head()

,Country.Code,Time,DT.NFL.UNAI.CD,DT.ODA.OATL.CD,DC.DAC.USAL.CD,DC.DAC.TOTL.CD,NY.GDP.MKTP.CD,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.CD,SP.POP.DPND.YG,...,SL.TLF.CACT.MA.ZS,SP.POP.TOTL,SL.UEM.TOTL.ZS,Country.Name,Time.Code,SH.STA.MMRT,SP.DYN.IMRT.IN,SP.DYN.LE00.FE.IN,SP.DYN.LE00.MA.IN,SP.DYN.LE00.IN
0,AFG,1960.0,NaN,NaN,15000000,15670000,537777811.1,NaN,59.77732651,76.52247462,...,NaN,8996351,NaN,Afghanistan,YR1960,NaN,245.7,33.133,31.58,32.33756098
1,AFG,1961.0,NaN,NaN,30000000,33550000,548888895.6,NaN,59.87815281,77.52917821,...,NaN,9166764,NaN,Afghanistan,YR1961,NaN,241.2,33.586,32.026,32.78697561
2,AFG,1962.0,NaN,NaN,13000000,15260000,546666677.8,NaN,58.49287383,78.08540637,...,NaN,9345868,NaN,Afghanistan,YR1962,NaN,236.9,34.03,32.466,33.22892683
3,AFG,1963.0,NaN,NaN,32000000,34880000,751111191.1,NaN,78.78275804,78.37535115,...,NaN,9533954,NaN,Afghanistan,YR1963,NaN,232.7,34.469,32.903,33.66690244
4,AFG,1964.0,NaN,NaN,37000000,44220000,800000044.4,NaN,82.20844386,78.62537426,...,NaN,9731361,NaN,Afghanistan,YR1964,NaN,228.5,34.904,33.338,34.10190244


### Merge WDI data with Codes data to start Main DataFrame

In [13]:
df = wdi.merge(codes, left_on='Country.Code', right_on='WBcode', how='left')
# remove redundant columns
del df['Country.Code']
del df['Country.Name']

In [14]:
df.head()

,Time,DT.NFL.UNAI.CD,DT.ODA.OATL.CD,DC.DAC.USAL.CD,DC.DAC.TOTL.CD,NY.GDP.MKTP.CD,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.CD,SP.POP.DPND.YG,SH.CON.1524.FE.ZS,...,SP.DYN.LE00.FE.IN,SP.DYN.LE00.MA.IN,SP.DYN.LE00.IN,WBcode,WBname,COWabb,COWcode,COWname,Income,MENA
0,1960.0,NaN,NaN,15000000,15670000,537777811.1,NaN,59.77732651,76.52247462,NaN,...,33.133,31.58,32.33756098,AFG,Afghanistan,AFG,700.0,Afghanistan,Low,0.0
1,1961.0,NaN,NaN,30000000,33550000,548888895.6,NaN,59.87815281,77.52917821,NaN,...,33.586,32.026,32.78697561,AFG,Afghanistan,AFG,700.0,Afghanistan,Low,0.0
2,1962.0,NaN,NaN,13000000,15260000,546666677.8,NaN,58.49287383,78.08540637,NaN,...,34.03,32.466,33.22892683,AFG,Afghanistan,AFG,700.0,Afghanistan,Low,0.0
3,1963.0,NaN,NaN,32000000,34880000,751111191.1,NaN,78.78275804,78.37535115,NaN,...,34.469,32.903,33.66690244,AFG,Afghanistan,AFG,700.0,Afghanistan,Low,0.0
4,1964.0,NaN,NaN,37000000,44220000,800000044.4,NaN,82.20844386,78.62537426,NaN,...,34.904,33.338,34.10190244,AFG,Afghanistan,AFG,700.0,Afghanistan,Low,0.0


### Polity IV Data on Regime Type (Autocracy - Democracy) and Durability
http://www.systemicpeace.org/inscrdata.html

In [17]:
polity = pd.read_excel('p4v2016.xls')
polity = polity[['ccode', 'year', 'polity2', 'durable']]
# polity2 = time series version of polity score -10, most autocratic, to 10, most democratic
# durable = number of years since a country has had a regime change (change of 3 or more in polity2)

In [18]:
polity.head()

,ccode,year,polity2,durable
0,700,1800,-6.0,NaN
1,700,1801,-6.0,NaN
2,700,1802,-6.0,NaN
3,700,1803,-6.0,NaN
4,700,1804,-6.0,NaN


### Merge with Main DataFrame

In [19]:
df = df.merge(polity, left_on=['COWcode', 'Time'], right_on=['ccode', 'year'], how='left')
del df['ccode']
del df['Time']

In [20]:
df.head()

,DT.NFL.UNAI.CD,DT.ODA.OATL.CD,DC.DAC.USAL.CD,DC.DAC.TOTL.CD,NY.GDP.MKTP.CD,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.CD,SP.POP.DPND.YG,SH.CON.1524.FE.ZS,SH.CON.1524.MA.ZS,...,WBcode,WBname,COWabb,COWcode,COWname,Income,MENA,year,polity2,durable
0,NaN,NaN,15000000,15670000,537777811.1,NaN,59.77732651,76.52247462,NaN,NaN,...,AFG,Afghanistan,AFG,700,Afghanistan,Low,0.0,1960.0,-10.0,NaN
1,NaN,NaN,30000000,33550000,548888895.6,NaN,59.87815281,77.52917821,NaN,NaN,...,AFG,Afghanistan,AFG,700,Afghanistan,Low,0.0,1961.0,-10.0,NaN
2,NaN,NaN,13000000,15260000,546666677.8,NaN,58.49287383,78.08540637,NaN,NaN,...,AFG,Afghanistan,AFG,700,Afghanistan,Low,0.0,1962.0,-10.0,NaN
3,NaN,NaN,32000000,34880000,751111191.1,NaN,78.78275804,78.37535115,NaN,NaN,...,AFG,Afghanistan,AFG,700,Afghanistan,Low,0.0,1963.0,-10.0,NaN
4,NaN,NaN,37000000,44220000,800000044.4,NaN,82.20844386,78.62537426,NaN,NaN,...,AFG,Afghanistan,AFG,700,Afghanistan,Low,0.0,1964.0,-7.0,0.0


### USAID Donor/Recipient Data
http://aiddata.org/data/aiddata-core-research-release-level-1-3-1

In [22]:
aid = pd.read_csv(os.path.join('AidData', 'AidDataCoreDonorRecipientYearPurpose_ResearchRelease_Level1_v3.1.csv'))

##### Subset to aid delivered either by the UN or the US

In [23]:
aid.donor.value_counts()

United States                                                                             56631
Germany                                                                                   51193
France                                                                                    50875
Spain                                                                                     38655
Canada                                                                                    35829
Norway                                                                                    31935
Japan                                                                                     31220
United Kingdom                                                                            29545
United Nations Children`s Fund (UNICEF)                                                   29254
Belgium                                                                                   28236
Netherlands                             

In [24]:
aid = aid[(aid['donor'] == 'United States') | (aid['donor'].str.contains('United Nations'))]
aid.donor.value_counts()

United States                                                                             56631
United Nations Children`s Fund (UNICEF)                                                   29254
United Nations Development Programme (UNDP)                                               14802
United Nations Population Fund (UNFPA)                                                     5836
Joint United Nations Programme on HIV/AIDS (UNAIDS)                                        2102
United Nations Economic Commission for Europe (UNECE)                                       232
United Nations Democracy Fund (UNDEF)                                                       220
United Nations High Commissioner for Refugees (UNHCR)                                       158
United Nations Peacebuilding Fund (UNPBF)                                                   151
United Nations Relief and Works Agency for Palestine Refugees in the Near East (UNRWA)       48
United Nations Economic and Social Commi

In [25]:
# aggregate
aid['donor_agg'] = 'United Nations'
aid.loc[aid['donor']=='United States', 'donor_agg'] = 'United States'
aid.donor_agg.value_counts()

United States     56631
United Nations    52809
Name: donor_agg, dtype: int64

##### Focus on aid impacting women, reproductive rights, and family planning

In [26]:
for k, v in aid.coalesced_purpose_name.value_counts().items():
    print(k, '-', v)

Strengthening civil society - 4405
Multisector aid - 4120
Social/ welfare services - 3564
Reproductive health care - 3416
STD control including HIV/AIDS - 3008
Economic and development policy/planning - 2995
Std control including hiv/aids - 2969
Legal and judicial development - 2633
Government administration - 2478
Basic health care - 2325
Material relief assistance and services - 2304
Sectors not specified - 2258
Primary education - 2257
Population policy and administrative management - 2148
Family planning - 1706
Food security programmes/food aid - 1582
Multisector aid for social services - 1577
Basic nutrition - 1549
Environmental policy and administrative management - 1543
Business support services and institutions - 1411
Disaster prevention and preparedness - 1371
Population policy and admin. mgmt - 1313
Agricultural development - 1295
Bio-diversity - 1206
Infectious disease control - 1142
Employment policy and administrative management - 1138
Administrative costs - 1091
Industria

####### Categories of note
- Reproductive health care - 3416
- STD control including HIV/AIDS - 3008
- Std control including hiv/aids - 2969
- Family planning - 1706
- Women in development - 771
- Women's equality organisations and institutions - 169
- Health, combination of general, basic, and population policy/reproductive health purposes - 136
- Population policies/ programmes and reproductive health, combinations of activities - 135
- FAMILY PLANNING 12
- WOMEN'S EQUALITY ORGANISATIONS AND INSTITUTIONS 1

In [27]:
aid['aid_focus'] = 'Other'
aid.loc[aid['coalesced_purpose_name'] == 'Reproductive health care', 'aid_focus'] = 'Reproductive health care'
aid.loc[(aid['coalesced_purpose_name'] == 'STD control including HIV/AIDS') |
        (aid['coalesced_purpose_name'] == 'Std control including hiv/aids'), 'aid_focus'] = 'STD/HIV/AIDS'
aid.loc[(aid['coalesced_purpose_name'] == 'Family planning') |
        (aid['coalesced_purpose_name'] == 'FAMILY PLANNING'), 'aid_focus'] = 'Family planning'
aid.loc[aid['coalesced_purpose_name'] == 'Women in development', 'aid_focus'] = 'Women in development'
aid.loc[(aid['coalesced_purpose_name'] == "Women's equality organisations and institutions") |
        (aid['coalesced_purpose_name'] == "WOMEN'S EQUALITY ORGANISATIONS AND INSTITUTIONS"), 'aid_focus'] = \
        'Womens equality organizations and institutions'
aid.loc[(aid['coalesced_purpose_name'] == 'Health, combination of general, basic, and population policy/reproductive health purposes') |
        (aid['coalesced_purpose_name'] == "Population policies/ programmes and reproductive health, combinations of activities"), 'aid_focus'] = \
        'General health including reproductive health'
aid.aid_focus.value_counts()

Other                                             97117
STD/HIV/AIDS                                       5977
Reproductive health care                           3416
Family planning                                    1718
Women in development                                771
General health including reproductive health        271
Womens equality organizations and institutions      170
Name: aid_focus, dtype: int64

In [28]:
aid.columns

Index(['donor', 'recipient', 'year', 'coalesced_purpose_code',
       'coalesced_purpose_name', 'commitment_amount_usd_constant_sum',
       'donor_agg', 'aid_focus'],
      dtype='object')

##### Make wide (country-year format)

In [29]:
aid_donor_agg = aid.pivot_table(index=['recipient', 'year'], columns='donor_agg', 
                       values='commitment_amount_usd_constant_sum', aggfunc=sum).reset_index()
aid_donor = aid.pivot_table(index=['recipient', 'year'], columns='donor', 
                       values='commitment_amount_usd_constant_sum', aggfunc=sum).reset_index()
del aid_donor['United States']
aid_focus = aid.pivot_table(index=['recipient', 'year'], columns='aid_focus', 
                       values='commitment_amount_usd_constant_sum', aggfunc=sum).reset_index()
aid_focus['Womens Aid'] = aid_focus[['STD/HIV/AIDS', 'Reproductive health care', 'Family planning', 
                                 'Women in development', 'General health including reproductive health', 
                                 'Womens equality organizations and institutions']].sum(axis=1)
aid_donor_focus = aid.pivot_table(index=['recipient', 'year'], columns=['donor_agg', 'aid_focus'], 
                       values='commitment_amount_usd_constant_sum', aggfunc=sum).reset_index()
aid_donor_focus.columns = ['{}{}'.format(a, '{}'.format('-'+b if b else '')) for a, b in aid_donor_focus.columns]
un_cols = [col for col in aid_donor_focus if 'United Nations' in col and 'Other' not in col]
aid_donor_focus['United Nations-Womens Aid'] = aid_donor_focus[un_cols].sum(axis=1)
us_cols = [col for col in aid_donor_focus if 'United States' in col and 'Other' not in col]
aid_donor_focus['United States-Womens Aid'] = aid_donor_focus[us_cols].sum(axis=1)
aid_wide = aid_donor_agg.merge(aid_donor, on=['recipient', 'year'], 
                               how='outer').merge(aid_focus, on=['recipient', 'year'], how='outer') \
                                .merge(aid_donor_focus, on=['recipient', 'year'], how='outer')

In [30]:
aid_wide.fillna(0, inplace=True)

In [31]:
aid_wide.head()

,recipient,year,United Nations,United States,Joint United Nations Programme on HIV/AIDS (UNAIDS),United Nations Children`s Fund (UNICEF),United Nations Democracy Fund (UNDEF),United Nations Development Programme (UNDP),United Nations Economic Commission for Europe (UNECE),United Nations Economic and Social Commission for Asia and the Pacific (UNESCAP),...,United Nations-Womens equality organizations and institutions,United States-Family planning,United States-General health including reproductive health,United States-Other,United States-Reproductive health care,United States-STD/HIV/AIDS,United States-Women in development,United States-Womens equality organizations and institutions,United Nations-Womens Aid,United States-Womens Aid
0,Afghanistan,1973,0.0,40767666.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,40767666.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,1974,0.0,35999152.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,35999152.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,1975,0.0,27871564.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27871564.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,1976,0.0,8688297.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8688297.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,1977,0.0,16124956.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,16124956.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Map recipient to country names from main data

In [32]:
recip_map_list = []
for recip in aid.recipient.unique():
    recip_dict = {'AIDname': recip}
    if recip in codes.WBname.tolist():
        recip_dict['WBname'] = codes.loc[codes['WBname']==recip, 'WBname'].values[0]
    else:
        recip_dict['WBname'] = None
    if recip in codes.COWname.tolist():
        recip_dict['COWname'] = codes.loc[codes['COWname']==recip, 'COWname'].values[0]
    else:
        recip_dict['COWname'] = None
    recip_map_list.append(recip_dict)
recip_map = pd.DataFrame(recip_map_list)

In [33]:
recip_map = recip_map.loc[~((recip_map.AIDname.str.contains('Regional')) | 
                            (recip_map.AIDname.str.contains('Bilateral')) | 
                            (recip_map.AIDname.str.contains('Unspecified')) |
                            (recip_map.AIDname.str.contains('Multi')) |
                            (recip_map.AIDname == 'no value') |
                            (recip_map.AIDname == 'Global') |
                            (recip_map.AIDname == 'European Commission'))]

In [34]:
recip_map[recip_map[['COWname', 'WBname']].isnull().all(axis=1)]

,AIDname,COWname,WBname
21,Bosnia-Herzegovina,None,None
36,"Congo, Democratic Republic of",None,None
37,"Congo, Republic of",None,None
39,Cote d`Ivoire,None,None
73,"Korea, Democratic Republic of",None,None
104,Niue,None,None
117,Sao Tome & Principe,None,None
128,South Sudan,None,None
130,St. Kitts & Nevis,None,None
132,St.Vincent & Grenadines,None,None


In [35]:
manual_map_list = [
    {
    'AIDname': 'Congo, Democratic Republic of',
    'WBname': 'Congo, Dem. Rep.',
    'COWname': 'Democratic Republic of the Congo'
    }, 
    {
    'AIDname': 'Congo, Republic of',
    'WBname': 'Congo, Rep.',
    'COWname': 'Congo'
    }, 
    {
    'AIDname': 'Cote d`Ivoire',
    'WBname': "Cote d'Ivoire",
    'COWname': 'Ivory Coast'
    },
    {
    'AIDname': 'Sao Tome & Principe',
    'WBname': 'Sao Tome and Principe',
    'COWname': 'Sao Tome and Principe'
    }, 
    {
    'AIDname': 'Micronesia',
    'WBname': 'Micronesia, Fed. Sts.',
    'COWname': 'Federated States of Micronesia'
    }, 
    {
    'AIDname': 'Viet Nam',
    'WBname': 'Vietnam',
    'COWname': 'Vietnam'
    }, 
    {
    'AIDname': 'Bosnia-Herzegovina',
    'WBname': 'Bosnia and Herzegovina',
    'COWname': 'Bosnia and Herzegovina'
    }, 
    {
    'AIDname': 'Korea, Democratic Republic of',
    'WBname': 'Korea, Democratic Republic of',
    'COWname': 'North Korea'
    }, 
    {
    'AIDname': 'St. Kitts & Nevis',
    'WBname': 'St. Kitts and Nevis',
    'COWname': 'St. Kitts and Nevis'
    }, 
    {
    'AIDname': 'St.Vincent & Grenadines',
    'WBname': 'St. Vincent and the Grenadines',
    'COWname': 'St. Vincent and the Grenadines'
    }, 
    {
    'AIDname': 'Trinidad & Tobago',
    'WBname': 'Trinidad and Tobago',
    'COWname': 'Trinidad and Tobago'
    }
]
manual_map = pd.DataFrame(manual_map_list)
manual_map

,AIDname,COWname,WBname
0,"Congo, Democratic Republic of",Democratic Republic of the Congo,"Congo, Dem. Rep."
1,"Congo, Republic of",Congo,"Congo, Rep."
2,Cote d`Ivoire,Ivory Coast,Cote d'Ivoire
3,Sao Tome & Principe,Sao Tome and Principe,Sao Tome and Principe
4,Micronesia,Federated States of Micronesia,"Micronesia, Fed. Sts."
5,Viet Nam,Vietnam,Vietnam
6,Bosnia-Herzegovina,Bosnia and Herzegovina,Bosnia and Herzegovina
7,"Korea, Democratic Republic of",North Korea,"Korea, Democratic Republic of"
8,St. Kitts & Nevis,St. Kitts and Nevis,St. Kitts and Nevis
9,St.Vincent & Grenadines,St. Vincent and the Grenadines,St. Vincent and the Grenadines


In [36]:
for i, row in manual_map.iterrows():
    recip_map.loc[recip_map['AIDname']==row['AIDname'], 'COWname'] = row['COWname']
    recip_map.loc[recip_map['AIDname']==row['AIDname'], 'WBname'] = row['WBname']

In [37]:
aid_wide = aid_wide.merge(recip_map, left_on='recipient', right_on='AIDname', how='left')
aid_wide = aid_wide.loc[~aid_wide[['COWname', 'WBname']].isnull().all(axis=1)]
aid_wide.head()

,recipient,year,United Nations,United States,Joint United Nations Programme on HIV/AIDS (UNAIDS),United Nations Children`s Fund (UNICEF),United Nations Democracy Fund (UNDEF),United Nations Development Programme (UNDP),United Nations Economic Commission for Europe (UNECE),United Nations Economic and Social Commission for Asia and the Pacific (UNESCAP),...,United States-Other,United States-Reproductive health care,United States-STD/HIV/AIDS,United States-Women in development,United States-Womens equality organizations and institutions,United Nations-Womens Aid,United States-Womens Aid,AIDname,COWname,WBname
0,Afghanistan,1973,0.0,40767666.0,0.0,0.0,0.0,0.0,0.0,0.0,...,40767666.0,0.0,0.0,0.0,0.0,0.0,0.0,Afghanistan,Afghanistan,Afghanistan
1,Afghanistan,1974,0.0,35999152.0,0.0,0.0,0.0,0.0,0.0,0.0,...,35999152.0,0.0,0.0,0.0,0.0,0.0,0.0,Afghanistan,Afghanistan,Afghanistan
2,Afghanistan,1975,0.0,27871564.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27871564.0,0.0,0.0,0.0,0.0,0.0,0.0,Afghanistan,Afghanistan,Afghanistan
3,Afghanistan,1976,0.0,8688297.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8688297.0,0.0,0.0,0.0,0.0,0.0,0.0,Afghanistan,Afghanistan,Afghanistan
4,Afghanistan,1977,0.0,16124956.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16124956.0,0.0,0.0,0.0,0.0,0.0,0.0,Afghanistan,Afghanistan,Afghanistan


In [38]:
del aid_wide['recipient']

### Merge with Main DataFrame

In [39]:
# first on COWname, then WBname
df1 = df.merge(aid_wide, on=['COWname', 'year'], how='left')
del df1['WBname_y']
df1.rename(columns={'WBname_x': 'WBname'}, inplace=True)
df2 = df.merge(aid_wide, on=['WBname', 'year'], how='left')
del df2['COWname_y']
df2.rename(columns={'COWname_x': 'COWname'}, inplace=True)

In [40]:
df1.columns

Index(['DT.NFL.UNAI.CD', 'DT.ODA.OATL.CD', 'DC.DAC.USAL.CD', 'DC.DAC.TOTL.CD',
       'NY.GDP.MKTP.CD', 'NY.GDP.MKTP.KD.ZG', 'NY.GDP.PCAP.CD',
       'SP.POP.DPND.YG', 'SH.CON.1524.FE.ZS', 'SH.CON.1524.MA.ZS',
       'per_si_allsi.cov_pop_tot', 'per_sa_allsa.cov_pop_tot',
       'per_lm_alllm.cov_pop_tot', 'SE.SEC.CUAT.LO.ZS', 'SE.SEC.CUAT.LO.FE.ZS',
       'SE.SEC.CUAT.LO.MA.ZS', 'SE.PRM.CUAT.FE.ZS', 'SE.PRM.CUAT.MA.ZS',
       'SE.PRM.CUAT.ZS', 'SH.HIV.INCD.ZS', 'IT.NET.USER.ZS',
       'SL.TLF.CACT.FE.ZS', 'SL.TLF.CACT.MA.ZS', 'SP.POP.TOTL',
       'SL.UEM.TOTL.ZS', 'Time.Code', 'SH.STA.MMRT', 'SP.DYN.IMRT.IN',
       'SP.DYN.LE00.FE.IN', 'SP.DYN.LE00.MA.IN', 'SP.DYN.LE00.IN', 'WBcode',
       'WBname', 'COWabb', 'COWcode', 'COWname', 'Income', 'MENA', 'year',
       'polity2', 'durable', 'United Nations', 'United States',
       'Joint United Nations Programme on HIV/AIDS (UNAIDS)',
       'United Nations Children`s Fund (UNICEF)',
       'United Nations Democracy Fund (UNDEF)',
  

In [41]:
df1.columns == df2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [42]:
df3 = pd.concat([df1, df2])

In [43]:
df = df3.drop_duplicates()
print(len(df3), len(df))

26422 12927


In [44]:
df.head()

,DT.NFL.UNAI.CD,DT.ODA.OATL.CD,DC.DAC.USAL.CD,DC.DAC.TOTL.CD,NY.GDP.MKTP.CD,NY.GDP.MKTP.KD.ZG,NY.GDP.PCAP.CD,SP.POP.DPND.YG,SH.CON.1524.FE.ZS,SH.CON.1524.MA.ZS,...,United States-Family planning,United States-General health including reproductive health,United States-Other,United States-Reproductive health care,United States-STD/HIV/AIDS,United States-Women in development,United States-Womens equality organizations and institutions,United Nations-Womens Aid,United States-Womens Aid,AIDname
0,NaN,NaN,15000000,15670000,537777811.1,NaN,59.77732651,76.52247462,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000000,33550000,548888895.6,NaN,59.87815281,77.52917821,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,13000000,15260000,546666677.8,NaN,58.49287383,78.08540637,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,32000000,34880000,751111191.1,NaN,78.78275804,78.37535115,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,37000000,44220000,800000044.4,NaN,82.20844386,78.62537426,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df.columns

Index(['DT.NFL.UNAI.CD', 'DT.ODA.OATL.CD', 'DC.DAC.USAL.CD', 'DC.DAC.TOTL.CD',
       'NY.GDP.MKTP.CD', 'NY.GDP.MKTP.KD.ZG', 'NY.GDP.PCAP.CD',
       'SP.POP.DPND.YG', 'SH.CON.1524.FE.ZS', 'SH.CON.1524.MA.ZS',
       'per_si_allsi.cov_pop_tot', 'per_sa_allsa.cov_pop_tot',
       'per_lm_alllm.cov_pop_tot', 'SE.SEC.CUAT.LO.ZS', 'SE.SEC.CUAT.LO.FE.ZS',
       'SE.SEC.CUAT.LO.MA.ZS', 'SE.PRM.CUAT.FE.ZS', 'SE.PRM.CUAT.MA.ZS',
       'SE.PRM.CUAT.ZS', 'SH.HIV.INCD.ZS', 'IT.NET.USER.ZS',
       'SL.TLF.CACT.FE.ZS', 'SL.TLF.CACT.MA.ZS', 'SP.POP.TOTL',
       'SL.UEM.TOTL.ZS', 'Time.Code', 'SH.STA.MMRT', 'SP.DYN.IMRT.IN',
       'SP.DYN.LE00.FE.IN', 'SP.DYN.LE00.MA.IN', 'SP.DYN.LE00.IN', 'WBcode',
       'WBname', 'COWabb', 'COWcode', 'COWname', 'Income', 'MENA', 'year',
       'polity2', 'durable', 'United Nations', 'United States',
       'Joint United Nations Programme on HIV/AIDS (UNAIDS)',
       'United Nations Children`s Fund (UNICEF)',
       'United Nations Democracy Fund (UNDEF)',
  

In [46]:
df.to_csv('full_data.csv', index=False)

In [47]:
wdi_col_map = pd.DataFrame.from_dict(wdi_col_map, orient='index').reset_index()
wdi_col_map.columns = ['col_name', 'description']
wdi_col_map.to_csv('wdi_column_name_mapping.csv', index=False)